In [1]:
import tensorflow as tf
tf.config.experimental.set_visible_devices([], 'GPU')

2025-03-01 16:41:33.821245: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-01 16:41:33.831930: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740832893.845593  346930 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740832893.849885  346930 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-01 16:41:33.864138: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
from model import (
    TransformerConfig,
    TransformerLM
)
from trainer import (
    create_learning_rate_schedule,
    TrainState,
    _to_array,
    compute_weighted_cross_entropy,
    compute_metrics
)
from steps import train_step
import optax
import jax
import jax.numpy as jnp
from flax import nnx
from jax import random
from clu import metric_writers, periodic_actions
from absl import logging
from flax.training import common_utils

In [3]:
learning_rate_fn = create_learning_rate_schedule(
    learning_rate=0.0016, warmup_steps=100
)
tx = optax.adamw(learning_rate_fn, b1=0.9, b2=0.98, eps=1e-9, weight_decay=0.1)

config = TransformerConfig(vocab_size=30728)
model = TransformerLM(config, rngs=nnx.Rngs(0))

graphdef, params, rest = nnx.split(model, nnx.Param, ...)
state = TrainState.create(
    apply_fn=graphdef.apply, params=params, tx=tx, graphdef=graphdef, rest=rest
)

state = jax.tree.map(_to_array, state)


In [4]:
jit_train_step = jax.jit(train_step, static_argnums=(2, 3), donate_argnums=0)

In [5]:
start_step = 0
rng = jax.random.PRNGKey(42)
rng, init_rng = jax.random.split(rng)
rng, inference_rng = random.split(rng)

In [6]:
writer = metric_writers.create_default_writer(
    './', just_logging=jax.process_index() > 0
)

In [7]:
from tqdm import tqdm

In [8]:
dropout_rngs = rng

logging.info('Starting training loop.')
hooks = []

report_progress = periodic_actions.ReportProgress(
    num_train_steps=10, writer=writer
)

if jax.process_index() == 0:
    hooks += [
        report_progress,
        periodic_actions.Profile(logdir='./', num_profile_steps=5),
    ]

train_metrics = []

with metric_writers.ensure_flushes(writer):
    for step in tqdm(range(0, 100_000)):
        with jax.profiler.StepTraceAnnotation('train', step_num=step):
            rng, _ = jax.random.split(rng)
            batch = jax.random.randint(rng, (3, 64), minval=0, maxval=70_000)
            state, metrics = jit_train_step(
                state, batch, learning_rate_fn, 0.0, dropout_rngs
            )
            train_metrics.append(metrics)

        # Quick indication that training is happening.
        logging.log_first_n(logging.INFO, 'Finished training step %d.', 5, step)
        for h in hooks:
            h(step)

        if step % 10 == 0:
            with report_progress.timed('training_metrics'):
                logging.info('Gathering training metrics.')
                train_metrics = common_utils.stack_forest(train_metrics)
                lr = train_metrics.pop('learning_rate').mean()
                metrics_sums = jax.tree.map(jnp.sum, train_metrics)
                denominator = metrics_sums.pop('denominator')
                summary = jax.tree.map(lambda x: x / denominator, metrics_sums)  # pylint: disable=cell-var-from-loop
                summary['learning_rate'] = lr
                summary['perplexity'] = jnp.clip(jnp.exp(summary['loss']), max=1.0e4)
                summary = {'train_' + k: v for k, v in summary.items()}
                writer.write_scalars(step, summary)
                train_metrics = []


        

  6%|▌         | 5747/100000 [01:06<18:06, 86.78it/s] 


KeyboardInterrupt: 